### 최종

In [130]:
### 0. import
# crawling 관련
import time
import random
from bs4 import BeautifulSoup
from selenium import webdriver

# db insert 관련
import psycopg2
import numpy  as np
import pandas as pd
from datetime import datetime


### 1. 법정동명 가져오기 (법정동 18,861개)
# AWS DW Connect
aws_conn = psycopg2.connect(
    host = 'redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
    port = 5439,
    dbname = 'dev',
    user = 'awsuser',
    password = 'cremaoAdmin1234qwer!!'
)
aws_cursor = aws_conn.cursor()

sql = "select distinct emd_cd, emd_nm from m1.pnu"
aws_cursor.execute(sql)
aws_conn.commit()
rows = aws_cursor.fetchall()

df_emd = pd.DataFrame(rows)
df_emd.columns = ['emd_cd','emd_full_nm']
df_emd = df_emd[~df_emd['emd_full_nm'].isnull()] # emd_nm이 널인 지역 제외
df_emd['emd_nm'] = df_emd['emd_full_nm'].str.split(' ').str[-1] # 동이름만 추출

### 1-1. 동일한 이름의 지역 따로 처리
# 동이름 같은 지역 추출
cd_list = []
for i in df_emd.index:
    if (len(df_emd[df_emd['emd_nm']==df_emd['emd_nm'][i]]) > 1) == True:
        cd_list.append(df_emd['emd_cd'][i])
        
# (시군구 + 법정동) 명으로 변경
for i in range(len(cd_list)):
    idx = df_emd[df_emd['emd_cd']==cd_list[i]].index
    df_emd['emd_nm'][idx] = df_emd['emd_full_nm'][idx[0]].split(' ')[-2] + df_emd['emd_full_nm'][idx[0]].split(' ')[-1]

In [131]:
df_emd

,emd_cd,emd_full_nm,emd_nm
0,1111010100,서울특별시 종로구 청운동,청운동
1,1111010200,서울특별시 종로구 신교동,종로구신교동
2,1111010300,서울특별시 종로구 궁정동,궁정동
3,1111010400,서울특별시 종로구 효자동,종로구효자동
4,1111010500,서울특별시 종로구 창성동,종로구창성동
...,...,...,...
18851,4611015300,전라남도 목포시 경동2가,경동2가
18852,4773036000,경상북도 의성군 가음면,가음면
18853,4773041000,경상북도 의성군 단밀면,단밀면
18855,4812111200,경상남도 창원시 의창구 덕정동,의창구덕정동


In [135]:
driver.close()

In [134]:
### 2. 법정동별 게시물 수 수집 (1차 수집)
driver = webdriver.Chrome(executable_path='C:\oasis\git\pythonProject\업무 완료\driver\chromedriver.exe')
# driver = webdriver.Chrome(executable_path='C:\\big14\driver\chromedriver.exe')
cnt_list = []
etc_emd_nm = [] # 추가작업 법정동

for dong in df_emd['emd_nm']:
    
    url = f'https://www.instagram.com/explore/tags/{dong}/' # url로 태그 검색
    time.sleep(random.randrange(1,3))
    driver.get(url)
    time.sleep(random.randrange(3,5))
    
    try:
        text = driver.find_element_by_class_name('_ac2a').text # 전체 게시물 수 가져오기
        cnt_list.append(text)
        
    except:
        cnt = '0'
        cnt_list.append(cnt)
        etc_emd_nm.append(dong)
        time.sleep(random.randrange(3,5))
        
        pass

# 데이터프레임화
df_cnt_list = pd.DataFrame(cnt_list,columns=['cnt'])
df1 = pd.concat([df_emd,df_cnt_list],axis=1)

In [141]:
# null 인 지역 (43개)
df1_null = df1[df1['cnt'].isnull()]

# 0인 지역 (16038개)
df1_0 = df1[df1['cnt'] == '0']

In [143]:
df1 = df1[~df1['cnt'].isnull()]
df1 = df1[~(df1['cnt'] == '0')]

In [155]:
df1

,emd_cd,emd_full_nm,emd_nm,cnt
0,1111010100,서울특별시 종로구 청운동,청운동,"14,958"
2,1111010300,서울특별시 종로구 궁정동,궁정동,"3,685"
3,1111010400,서울특별시 종로구 효자동,종로구효자동,16
5,1111010700,서울특별시 종로구 적선동,적선동,"4,195"
6,1111010800,서울특별시 종로구 통인동,통인동,"28,103"
...,...,...,...,...
9106,4886033029,경상남도 산청군 생초면 평촌리,생초면평촌리,"24,511"
10196,2920014900,광주광역시 광산구 지죽동,지죽동,"17,770"
14048,4513035026,전라북도 군산시 대야면 죽산리,대야면죽산리,10
16231,4711125625,경상북도 포항시 남구 오천읍 갈평리,오천읍갈평리,3


###### test

In [13]:
DF = df1[~df1['cnt'].isnull()]

In [14]:
DF # 1차 수집된 동 1186개

,emd_cd,emd_full_nm,emd_nm,cnt
0,1111010600,서울특별시 종로구 통의동,통의동,"36,938"
1,1111011000,서울특별시 종로구 누하동,누하동,"23,358"
2,1111011100,서울특별시 종로구 옥인동,옥인동,"16,478"
3,1111011200,서울특별시 종로구 체부동,체부동,"12,022"
4,1111011400,서울특별시 종로구 내자동,내자동,"13,905"
...,...,...,...,...
1181,2726012600,대구광역시 수성구 가천동,가천동,"2,513"
1182,2726012700,대구광역시 수성구 대흥동,수성구대흥동,24
1183,2729010100,대구광역시 달서구 성당동,성당동,"37,248"
1184,2729010200,대구광역시 달서구 두류동,두류동,"63,731"


In [16]:
# 2차 수집할 지역
df = df1[df1['cnt'].isnull()]

In [17]:
df

,emd_cd,emd_full_nm,emd_nm,cnt
1186,2729010700,대구광역시 달서구 신당동,달서구신당동,NaN
1187,2729010900,대구광역시 달서구 장동,달서구장동,NaN
1188,2729011000,대구광역시 달서구 장기동,달서구장기동,NaN
1189,2729011100,대구광역시 달서구 용산동,달서구용산동,NaN
1190,2729011400,대구광역시 달서구 본리동,본리동,NaN
...,...,...,...,...
18856,4874034031,경상남도 창녕군 이방면 송곡리,이방면송곡리,NaN
18857,4884025025,경상남도 남해군 남해읍 선소리,남해읍선소리,NaN
18858,5011012400,제주특별자치도 제주시 외도이동,외도이동,NaN
18859,5011025035,제주특별자치도 제주시 한림읍 한수리,한수리,NaN


In [18]:
### 법정동별 게시물 수 수집 (2차 수집)
# 2차 수집할 지역
df = df1[df1['cnt'].isnull()]
cnt_list = [] # 초기화

for dong in df['emd_nm']:
    
    url = f'https://www.instagram.com/explore/tags/{dong}/' # url로 태그 검색
    time.sleep(1)
    driver.get(url)
    time.sleep(5)
    
    try:
        text = driver.find_element_by_class_name('_ac2a').text # 전체 게시물 수 가져오기
        cnt_list.append(text)
        
    except:
        cnt = '0'
        cnt_list.append(cnt)
        
        time.sleep(3)
        pass

# 데이터프레임화
df_cnt_list = pd.DataFrame(cnt_list,columns=['cnt_1'])
df2 = pd.concat([df1,df_cnt_list],axis=1)
# driver.close()

In [30]:
df2[df2['cnt_1'].isnull()]

,emd_cd,emd_full_nm,emd_nm,cnt,cnt_1
17675,4822033021,경상남도 통영시 도산면 원산리,도산면원산리,NaN,NaN
17676,4822033022,경상남도 통영시 도산면 도선리,도선리,NaN,NaN
17677,4822033023,경상남도 통영시 도산면 법송리,법송리,NaN,NaN
17678,4822033025,경상남도 통영시 도산면 오륜리,오륜리,NaN,NaN
17679,4822033026,경상남도 통영시 도산면 저산리,도산면저산리,NaN,NaN
...,...,...,...,...,...
18856,4874034031,경상남도 창녕군 이방면 송곡리,이방면송곡리,NaN,NaN
18857,4884025025,경상남도 남해군 남해읍 선소리,남해읍선소리,NaN,NaN
18858,5011012400,제주특별자치도 제주시 외도이동,외도이동,NaN,NaN
18859,5011025035,제주특별자치도 제주시 한림읍 한수리,한수리,NaN,NaN


In [41]:
DF = df2 # sub_save

In [90]:
DF

,emd_cd,emd_full_nm,emd_nm,cnt,cnt_1
0,1111010600,서울특별시 종로구 통의동,통의동,36938,29
1,1111011000,서울특별시 종로구 누하동,누하동,23358,0
2,1111011100,서울특별시 종로구 옥인동,옥인동,16478,223
3,1111011200,서울특별시 종로구 체부동,체부동,12022,286
4,1111011400,서울특별시 종로구 내자동,내자동,39950,39950
...,...,...,...,...,...
18856,4874034031,경상남도 창녕군 이방면 송곡리,이방면송곡리,0,0
18857,4884025025,경상남도 남해군 남해읍 선소리,남해읍선소리,0,0
18858,5011012400,제주특별자치도 제주시 외도이동,외도이동,0,0
18859,5011025035,제주특별자치도 제주시 한림읍 한수리,한수리,0,0


In [101]:
DF[DF['emd_full_nm'].isnull()]

,emd_cd,emd_full_nm,emd_nm,cnt,cnt_1
418,2771036030,None,None,0,0
419,2771037022,None,None,0,0
1660,3611012000,None,None,0,0
1661,3611012100,None,None,0,0
1662,3611012300,None,None,0,0
2089,4148039029,None,None,663,663
2093,4148041024,None,None,0,0
5198,3611011900,None,None,0,0
5199,3611012200,None,None,6,6
5522,4122035029,None,None,1841,1841


In [107]:
# 일단 DW 적재
df = DF[DF['cnt']!=0]

In [109]:
df = df.drop(columns=['cnt_1','emd_nm'])

In [119]:
# 컬럼명 변경
df = df.rename(columns={'emd_full_nm':'emd_nm'})

In [126]:
# emd_nm 널인 지역 제외
df = df[~df['emd_nm'].isnull()]

In [127]:
df

,emd_cd,emd_nm,cnt
0,1111010600,서울특별시 종로구 통의동,36938
1,1111011000,서울특별시 종로구 누하동,23358
2,1111011100,서울특별시 종로구 옥인동,16478
3,1111011200,서울특별시 종로구 체부동,12022
4,1111011400,서울특별시 종로구 내자동,39950
...,...,...,...
17660,4822025021,경상남도 통영시 산양읍 영운리,11
17666,4822025027,경상남도 통영시 산양읍 풍화리,18
17667,4822025028,경상남도 통영시 산양읍 곤리리,2926
17672,4822031024,경상남도 통영시 용남면 삼화리,196


In [69]:
# 3차수집 데이터프레임
df3 = df2[df2['cnt_1']=='nan']

In [70]:
### 수집한 값이 더 크면 값 대체
# nan to 0
idx_cnt = df2[df2['cnt']=='nan'].index
df2['cnt'][idx_cnt] = 0

idx_cnt_1 = df2[df2['cnt_1']=='nan'].index
df2['cnt_1'][idx_cnt_1] = 0

df2['cnt'] = df2['cnt'].astype(int)
df2['cnt_1'] = df2['cnt_1'].astype(int)

# 값 변경
for i in df2.index:
    if ((df2['cnt'][i] < df2['cnt_1'][i]) == True):
        df2['cnt'][i] = df2['cnt_1'][i]

In [77]:
# cnt_1 삭제
# df2.drop(columns=['cnt_1'])

,emd_cd,emd_full_nm,emd_nm,cnt
0,1111010600,서울특별시 종로구 통의동,통의동,36938
1,1111011000,서울특별시 종로구 누하동,누하동,23358
2,1111011100,서울특별시 종로구 옥인동,옥인동,16478
3,1111011200,서울특별시 종로구 체부동,체부동,12022
4,1111011400,서울특별시 종로구 내자동,내자동,39950
...,...,...,...,...
18856,4874034031,경상남도 창녕군 이방면 송곡리,이방면송곡리,0
18857,4884025025,경상남도 남해군 남해읍 선소리,남해읍선소리,0
18858,5011012400,제주특별자치도 제주시 외도이동,외도이동,0
18859,5011025035,제주특별자치도 제주시 한림읍 한수리,한수리,0


In [84]:
df2[df2['cnt']==0]

,emd_cd,emd_full_nm,emd_nm,cnt,cnt_1
18,1111016000,서울특별시 종로구 연지동,종로구연지동,0,0
32,1114010700,서울특별시 중구 삼각동,중구삼각동,0,0
38,1114011900,서울특별시 중구 봉래동1가,중구봉래동1가,0,0
50,1114015200,서울특별시 중구 주교동,중구주교동,0,0
141,1144011700,서울특별시 마포구 신정동,마포구신정동,0,0
...,...,...,...,...,...
18856,4874034031,경상남도 창녕군 이방면 송곡리,이방면송곡리,0,0
18857,4884025025,경상남도 남해군 남해읍 선소리,남해읍선소리,0,0
18858,5011012400,제주특별자치도 제주시 외도이동,외도이동,0,0
18859,5011025035,제주특별자치도 제주시 한림읍 한수리,한수리,0,0


In [23]:
# 숫자 , 제외 / base_ym 컬럼 추가
df = df.drop(columns=['cnt_y'])
df = df.rename(columns={'cnt_x':'cnt'})
df['cnt'] = df['cnt'].str.replace(',','')
df['base_ym'] = datetime.today().strftime("%Y%m")

KeyboardInterrupt: 

기존 값이랑 매칭

In [193]:
DF2 = df2.drop(df2[df2['cnt_1'].isnull()].index)
DF2 = DF2.drop(DF2[DF2['cnt_1']=='0'].index)
DF2 = DF2.drop(columns=['cnt']).reset_index(drop=True)
DF2['cnt_1'] = DF2['cnt_1'].str.replace(',','')

In [221]:
# 수집된 값이 더 크면 업데이트 하기
for i in df2.index:
    idx = df_instargram[df2['emd_nm'][i] == df_instargram['emd_nm']].index[0]
    
    if df2['cnt_1'][i] > df_instargram['cnt'][idx]:
        df_instargram['cnt'][idx] = df2['cnt_1'][i]

In [35]:
df1 = df1.drop(df1[df1['emd_nm'].isnull()].index)
DF = df1[df1['cnt']>'0'].reset_index(drop=True)
DF['cnt'] = DF['cnt'].str.replace(',','')

In [217]:
# 수집된 값이 더 크면 업데이트 하기
for i in DF.index:
    idx = df_instargram[DF['emd_nm'][i] == df_instargram['emd_nm']].index[0]
    
    if DF['cnt'][i] > df_instargram['cnt'][idx]:
        df_instargram['cnt'][idx] = DF['cnt'][i]

재수집 목록 생성

In [134]:
# 재수집 목록 담기
idx = df1[df1['cnt']>'0'].index
df1 = df1.drop(idx).reset_index(drop=True)

In [186]:
### DB에서 값 받아와서 보충할 지역 재수집하기 (<0 or None인 지역만 재수집)
# AWS DW Connect
aws_conn = psycopg2.connect(
    host = 'redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
    port = 5439,
    dbname = 'dev',
    user = 'awsuser',
    password = 'cremaoAdmin1234qwer!!'
)
aws_cursor = aws_conn.cursor()

sql = "select * from m1.instagram_emd_cnt"
aws_cursor.execute(sql)
aws_conn.commit()
rows = aws_cursor.fetchall()

df_instargram = pd.DataFrame(rows)
df_instargram.columns = ['emd_nm','emd_cd','cnt','base_ym']
# df_emd = df_emd.drop_duplicates('emd_nm')
# df_emd['emd_cd'] = df_emd['emd_cd'] + '00'

In [187]:
df_instargram.sort_values('cnt', ascending=True)

,emd_nm,emd_cd,cnt,base_ym
2204,구호동,4217011500,0,202210
810,법성면,4687037000,0,202210
2415,대소원면,4313033500,0,202210
3932,황금동,2911011700,10,202210
1291,암곡동,4713013800,10,202210
...,...,...,...,...
2883,상망동,4721010200,99337,202210
2153,효제동,1111016200,9938,202210
2217,궁촌동,4418010600,996,202210
775,반여동,2635010300,99669,202210


In [119]:
df_instargram[df_instargram['cnt']<='0']

,emd_nm,emd_cd,cnt,base_ym
810,법성면,4687037000,0,202210
2204,구호동,4217011500,0,202210
2415,대소원면,4313033500,0,202210


In [230]:
df = df_instargram

In [128]:
### 3. DB INSERT
import datetime

table = 'm1.instagram_emd_cnt'

# execute_mogrify
def execute_mogrify(conn, df, table, val):
    # Create a list of tuples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL query to execute
    cursor = conn.cursor()
    values = [cursor.mogrify(val, tup).decode('utf8') for tup in
              tuples]
    query = "INSERT INTO %s(%s) VALUES " % (table, cols) + ",".join(values)

    try:
        cursor.execute(query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    cursor.close()

# values 설정
val = '(' + ('%s,' * len(df.columns))[:-1] + ')'

# insert
for j in range(0, len(df), 10000):

    # DB Connect
    conn = psycopg2.connect(
        host='redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
        port=5439,
        dbname='dev',
        user='awsuser',
        password='cremaoAdmin1234qwer!!'
    )

    # 10000개 단위로 나누기
    tmp = df[j:j + 10000]

    # insert
    execute_mogrify(conn, tmp, table, val)
    print(datetime.datetime.now(), ' : ', j)
    tmp = pd.DataFrame()

# row count
print(len(df))

2022-11-25 19:24:14.443821  :  0
5354


### 참고

In [45]:
### 받은 값 update
cnt_list = df1[df1['cnt']>'0']['cnt']
emd_cd_list = df1[df1['cnt']>'0']['emd_cd']

# AWS DW Connect
aws_conn = psycopg2.connect(
    host = 'redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
    port = 5439,
    dbname = 'dev',
    user = 'awsuser',
    password = 'cremaoAdmin1234qwer!!'
)
aws_cursor = aws_conn.cursor()

for cnt in cnt_list:
    for emd_cd in emd_cd_list:
        
        sql = f'''
        update m1.instagram_emd_cnt
           set cnt = '{cnt}'
         where emd_cd = '{emd_cd}'
        '''
        aws_cursor.execute(sql)
    
# aws_conn.commit()

KeyboardInterrupt: 

In [ ]:
### 로그인 하면 계정 차단됨
driver = webdriver.Chrome(executable_path='C:\oasis\git\pythonProject\업무 완료\driver\chromedriver.exe')
driver.get('http://www.instagram.com')
driver.maximize_window()
time.sleep(3)

email = 'oasis.0416'
input_id = driver.find_elements_by_xpath('//*[@id="loginForm"]/div/div[1]/div/label/input')[0]
input_id.clear()
input_id.send_keys(email)
password = 'localhouse'
input_pw = driver.find_elements_by_xpath('//*[@id="loginForm"]/div/div[2]/div/label/input')[0]
input_pw.clear()
input_pw.send_keys(password)
input_pw.submit()
time.sleep(3)

In [46]:
# 태현님 코드

from selenium import webdriver
from bs4 import BeautifulSoup
import time
import re
import random

def insta_searching(word):
    url = 'https://www.instagram.com/explore/tags/' + str(word)
    return url
def select_first(driver):
    first = driver.find_elements_by_css_selector('div._aagw')[0]
    first.click()
    time.sleep(3)
def get_content(driver):
    try:
        contents = driver.find_element_by_xpath('/html/body/div[5]/div[2]/div/article/div[3]/div[1]/ul/div/li/div/div/div[2]').text.split('\n')
    except:
        contents = ' '
    # 해시태그
    tags = []
    for C in contents:
        for c in C.split(' '):
            if '#' in c:
                tags.append(c)
    # 작성일
    date = contents[-1]
    data = [contents[0],date,tags]
    return data

def move_next(driver):
    right = driver.find_element_by_css_selector('a.coreSpriteRightPaginationArrow')
    right.click()
    time.sleep(3)
    
driver = webdriver.Chrome(executable_path='C:\oasis\git\pythonProject\업무 완료\driver\chromedriver.exe')
driver.get('http://www.instagram.com')
time.sleep(3)

# 로그인
email = 'th.park@ittang.co.kr'
input_id = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[0]
input_id.clear()
input_id.send_keys(email)
password = 'kindjoker12'
input_pw = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[1]
input_pw.clear()
input_pw.send_keys(password)
input_pw.submit()
time.sleep(5)

# 조회할 키워드
word = input('검색어')
word = str(word)
url = insta_searching(word)

# 결과페이지 열기
driver.get(url)
time.sleep(10)

# 첫 게시물 클릭
select_first(driver)
results = []
target = 3000

for i in range(target):
    try:
        time.sleep(random.randint(40,50))
        data = get_content(driver)
        results.append(data)
        move_next(driver)
    except:
        break
        
print(results[:2])

검색어 서귀포맛집


[[' ', ' ', []]]


In [30]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import re
import unicodedata
import pandas as pd



# 인스타그램 키워드 검색
def insta_searching(word):
    url = 'https://www.instagram.com/explore/tags/'+word
    return url

# 첫번째 요소 선택
def select_first(driver):
    time.sleep(5)
    first = driver.find_element_by_css_selector("div._aagw")
    first.click()
    time.sleep(3)

# 정보 가져오기
def get_content(driver):
    time.sleep(5)
    html = driver.page_source
    time.sleep(3)
    soup = BeautifulSoup(html,'html.parser')
    time.sleep(3)

    try:
        place = soup.select('div.M30cS')[0].text
        place = unicodedata.normalize('NFC',place)
        if place == '':
            data = ['', '', '', '', '']
            return data
    except:
        place = ''

    try:
        content = soup.select('div.C4VMK')[0].text
        content = unicodedata.normalize('NFC', content)
    except:
        content = ' '

    tags = re.findall(r'#[^\s#,\\]+', content)

    date = soup.select('time.FH9sR.Nzb55')[0]['datetime'][:10]

    try:
        like = soup.select('div.Nm9Fw')[0].text[4:-1]
    except:
        like = 0

    data = [content, date, like, place, tags]
    return data

def move_next(driver):
    right = driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow')
    right.click()
    time.sleep(3)

driver = webdriver.Chrome("C:\oasis\git\pythonProject\업무 완료\driver\chromedriver.exe")

# 인스타그램 접속
driver.get('https://www.instagram.com/accounts/login/')
time.sleep(2)



# 로그인
email = 'oasis.0416'
input_id = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[0]
input_id.clear()
input_id.send_keys(email)

password = 'localhouse'
input_pw = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[1]
input_pw.clear()
input_pw.send_keys(password)
input_pw.submit()
time.sleep(3)

# 키워드 검색
words = ['제주도맛집','제주맛집','제주도여행','제주여행']
for word in words:
    print(word)
    url = insta_searching(word)
    driver.get(url)
    time.sleep(2)

    # 검색 결과 첫 번째 선택
    select_first(driver)
    time.sleep(2)

    results = [ ]
    target = 300

    for i in range(target):
        try:
            # 내용 가져오기
            data = get_content(driver)
            if data[3] != '' :
                results.append(data)
            # 다음
            move_next(driver)
        except:
            time.sleep(2)
            move_next(driver)

    results_df = pd.DataFrame(results)
    results_df.columns = ['content','data','like','place','tags']

    if word == '제주도맛집':
        results_df.to_excel('./1_crawling_jejudoMatJib.xlsx',index=False)
    elif word == '제주맛집':
        results_df.to_excel('./1_crawling_jejuMatJib.xlsx',index=False)
    elif word == '제주도여행':
        results_df.to_excel('./1_crawling_jejudoYeoHang.xlsx',index=False)
    elif word == '제주여행':
        results_df.to_excel('./1_crawling_jejuYeoHang.xlsx',index=False)

제주도맛집


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"a._65Bje.coreSpriteRightPaginationArrow"}
  (Session info: chrome=106.0.5249.119)


In [ ]:
import pandas as pd

jeju_insta_df = pd.DataFrame([])

folder = './'
f_list = ['1_crawling_jejudoMatJib.xlsx',
          '1_crawling_jejuMatJib.xlsx',
          '1_crawling_jejudoYeoHang.xlsx',
          '1_crawling_jejuYeoHang.xlsx']

for frame in f_list:
    fpath = folder + frame
    temp = pd.read_excel(fpath)
    jeju_insta_df = jeju_insta_df.append(temp)

jeju_insta_df.columns = ['content','data','like','place','tags']

jeju_insta_df.drop_duplicates(subset = ['content'], inplace = True)
jeju_insta_df.to_excel('./2_crawling_raw.xlsx', index = False)

In [ ]:
import requests
import time
import pandas as pd


def find_places(searching):
    url = 'https://dapi.kakao.com/v2/local/search/keyword.json?query={}'.format(searching)
    headers = {
        "Authorization": "KakaoAK [Kakao API Key]"
    }
    places = requests.get(url, headers = headers).json()['documents']

    place = places[0]
    name = place['place_name']
    x = place['x']
    y = place['y']
    data = [name, x, y, searching]

    return data

raw_total = pd.read_excel('./2_crawling_raw.xlsx')
location_counts = raw_total['place'].value_counts()
location_counts_df = pd.DataFrame(location_counts)
location_counts_df.to_excel('./3_places.xlsx')

locations = list(location_counts.index)
locations_info = [ ]

for location in locations:
    try:
        data = find_places(location)
        locations_info.append(data)
        time.sleep(0.5)
    except:
        pass
# print(locations_info)

locations_info_df = pd.DataFrame(locations_info)
locations_info_df.columns = ['카카오위치명', '경도', '위도', '인스타위치명']
locations_info_df.to_excel('./4_hotplacelocations.xlsx', index=False)

location_counts_df = pd.read_excel('./3_places.xlsx', index_col=0)
locations_info_df = pd.read_excel('./4_hotplacelocations.xlsx')

location_data = pd.merge(locations_info_df, location_counts_df,
                         how = 'inner', left_on='인스타위치명', right_index=True)

# print(location_data)

location_data = location_data.pivot_table(
    index=['카카오위치명', '경도', '위도'],
    values='place',
    aggfunc='sum'
)
location_data.to_excel('./5_hotplaceslocations_merged.xlsx')

In [ ]:
import pandas as pd
import folium

location_data = pd.read_excel('./5_hotplaceslocations_merged.xlsx')

Hanla = [33.362500, 126.533694]
map_jeju = folium.Map(location=Hanla, zoom_start=11)

for i in range(len(location_data)):
    name = location_data['카카오위치명'][i]
    count = location_data['place'][i]
    size = int(count)*2
    long = float(location_data['위도'][i])
    lat = float(location_data['경도'][i])
    folium.CircleMarker((long,lat), radius=size, color='blue', popup=name).add_to(map_jeju)
    folium.Marker(
        location=[long,lat],
        popup=name,
    ).add_to(map_jeju)

map_jeju.save('./6_hotplacesmap.html')